In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from zipfile import ZipFile as zp
import torchvision.transforms as transforms

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import _LRScheduler
import torchvision

import torch.utils.data as data
import torch.nn.functional as F
from torchvision import models

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from tqdm.notebook import trange,tqdm_notebook   #tqdm is a library that is used for creating Python Progress Bars
from tqdm import tqdm
import time
import random
import numpy as np

In [3]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from zipfile import ZipFile as zp
# Load the dataset
zip_path = r"/content/cnn_dataset.zip"   # dynamic- change for online runs (colab/kaggle/aws)
data_path = r"images"

with zp(file=zip_path,mode="r") as obj :
    obj.extractall(path=data_path)
obj.close()

# Define the transformation to resize and convert the images to tensors
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.5125,0.4667,0.4110],
                         std= [0.2621,0.2501,0.2453])
])

# Load the dataset with normalization
dataset = ImageFolder(root=data_path, transform=transform)
print(dataset.classes)
print(len(dataset))

['dogs', 'food', 'vehicles']
30000


In [4]:
import torch.nn as nn

class VGG13(nn.Module):
    def __init__(self, num_classes=3):
        super(VGG13, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), 512 * 7 * 7)
        x = self.classifier(x)
        return x


In [5]:
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import random_split

# Split the dataset into training and testing sets
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create data loaders for the training and testing sets
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,num_workers = 2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,num_workers = 2)

In [ ]:
import torch
# Define the device to train on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Define the AlexNet model
model = VGG13(num_classes=3).to(device)
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
num_epochs = 10
for epoch in range(num_epochs):
    # Train the model on the training dataset
    model.train()
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
            # Evaluate the model on the testing dataset
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}: Test accuracy = {accuracy:.2f}%")

Epoch 1: Test accuracy = 32.50%
Epoch 2: Test accuracy = 32.52%
Epoch 3: Test accuracy = 32.52%
Epoch 4: Test accuracy = 32.50%
Epoch 5: Test accuracy = 32.52%
Epoch 6: Test accuracy = 34.98%
Epoch 7: Test accuracy = 34.98%
Epoch 8: Test accuracy = 32.52%
Epoch 9: Test accuracy = 32.52%
Epoch 10: Test accuracy = 34.98%


Here's an example of a modified VGG13 model that increases the number of channels and adds a dropout layer:

In [6]:
import torch.nn as nn

class ModifiedVGG13(nn.Module):
    def __init__(self, num_classes=3):
        super(ModifiedVGG13, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), 512 * 7 * 7)
        x = self.classifier(x)
        return x

# 새 섹션

In [ ]:
import torch
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ModifiedVGG13(num_classes=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
num_epochs = 10
for epoch in range(num_epochs):
    # Train the model on the training dataset
    model.train()
    for images, labels in tqdm(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
            # Evaluate the model on the testing dataset
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}: Test accuracy = {accuracy:.2f}%")

100%|██████████| 94/94 [00:01<00:00, 54.87it/s]


Epoch 1: Test accuracy = 43.10%


100%|██████████| 94/94 [00:01<00:00, 62.00it/s]


Epoch 2: Test accuracy = 78.52%


100%|██████████| 94/94 [00:01<00:00, 65.21it/s]


...

Epoch 8: Test accuracy = 91.95%


100%|██████████| 94/94 [00:01<00:00, 66.04it/s]


Epoch 9: Test accuracy = 90.45%


100%|██████████| 94/94 [00:01<00:00, 67.31it/s]

Epoch 10: Test accuracy = 92.58%


# Since we achieved acceptable performance of Test Accuracy 92.58% we are stopping at this point.

# Different modified models on the base model can be built like in part3 and 4 of the given assignment 2

In [ ]:
from tqdm import tqdm
import torch.optim as optim
import torch
# Check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = ModifiedVGG13(num_classes=3).to(device)
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Define the early stopping criteria
patience = 5  # Stop training if validation loss doesn't improve for 5 epochs
min_delta = 0.001  # Minimum change in validation loss to be considered an improvement
best_val_loss = float('inf')
counter = 0  # Counter for how many epochs the validation loss hasn't improved

# Train the model
for epoch in range(10):  # Train for 50 epochs or until early stopping criterion is met
    train_loss = 0.0
    test_loss = 0.0
    
    # Training loop
    model.train()
    for inputs, targets in tqdm(train_loader):
        optimizer.zero_grad()
        inputs, targets = inputs.to(device), targets.to(device)  # Move data to GPU
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
    
    # Testing loop
    model.eval()
    with torch.no_grad():
        for inputs, targets in tqdm(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)  # Move data to GPU
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item() * inputs.size(0)
    
    # Compute the average losses
    train_loss /= len(train_loader.dataset)
    test_loss /= len(test_loader.dataset)
    
    # Check if validation loss has improved
    if test_loss + min_delta < best_val_loss:
        best_val_loss = test_loss
        counter = 0
    else:
        counter += 1
    
    # Check if early stopping criteria have been met
    if counter >= patience:
        print(f'Test loss has not improved for {counter} epochs. Stopping training...')
        break
    
    print(f'Epoch {epoch + 1}: Train Loss = {train_loss:.4f}, Test Loss = {test_loss:.4f}')


100%|██████████| 94/94 [01:13<00:00,  1.28it/s]


Epoch 1: Train Loss = 1.0988, Test Loss = 1.0995


100%|██████████| 94/94 [01:05<00:00,  1.43it/s]


Epoch 2: Train Loss = 1.0543, Test Loss = 0.8429


100%|██████████| 94/94 [01:05<00:00,  1.43it/s]


...

Epoch 8: Train Loss = 0.2999, Test Loss = 0.3140


100%|██████████| 94/94 [01:06<00:00,  1.40it/s]


Epoch 9: Train Loss = 0.2773, Test Loss = 0.2830


100%|██████████| 94/94 [01:06<00:00,  1.42it/s]

Epoch 10: Train Loss = 0.2523, Test Loss = 0.2964
